In [ ]:
import math

# Define the stat sheet for offense
class Stats(object):
    def __init__(self, name, ws, bs, s, t, w, a, sv):
        self.name = name
        # required fields
        self.ws = float(ws)
        self.bs = float(bs)
        self.s = float(s)
        self.t = float(t)
        self.w = float(w)
        self.a = float(a)
        self.sv = float(sv)
        
        # optional fields
        self.ap = 0.0
        # wounds inflicted
        self.w_inf = 1.0
        # invulnerable save
        self.inv = 7.0
        self.has_inv = False
        
    def set_inv(self, inv):
        self.inv = inv
        self.has_inv = True
        
    def probability_to_hit(self, atk):
        return (7.0 - atk) / 6.0
    
    
    # str >= 2* tough => 2+
    # str > tough => 3+
    # str = tough => 4+
    # str < tough => 5+
    # 2*str <= tough => 6+
    def probability_to_wound(self, s, t):
        wound_on = 5.0
        if s >= 2*t:
            wound_on = 2.0
        elif s > t:
            wound_on = 3.0
        elif s == t:
            wound_on = 4.0
        elif 2*s < t:
            wound_on = 6.0
            
        return (7.0 - wound_on) / 6.0
    
    
    def probability_to_save(self, defender):
        # determine save
        save = defender.sv + self.ap
        if defender.has_inv:
            if defender.inv < save:
                save = defender.inv
                    
        return (7.0 - save) / 6.0
        
        
    def mean_attacks_to_kill(self, defender, is_ranged = False, s_ranged = 0.0, w_rnged = 0.0, ap_rng = 0.0):
        atk = self.ws
        s = self.s
        w_inf = self.w_inf
        ap = self.ap
        
        if is_ranged:
            atk = self.bs
            s = s_ranged
            w_inf = w_rnged
            ap = ap_rng
            
        # probability of hitting
        prob_hit = self.probability_to_hit(atk)
        
        # probability of wounding
        t = defender.t
        
        prob_wound = self.probability_to_wound(s, t)
        
        # probability of saving against the wound
        prob_save = self.probability_to_save(defender)
        
        attacks_to_wound = int(1.0 / (prob_hit * prob_wound * (1.0 - prob_save)))
        wound_to_kill = int(math.ceil(defender.w / self.w_inf))
        attacks_to_kill = attacks_to_wound * wound_to_kill
        print "Probability of " + self.name + " hitting " + defender.name + ": " + str(prob_hit) + "\nProbability of wounding: " + str(prob_wound) + "\nProbability of saving: " + str(prob_save) + "\nTotal attacks to kill: " + str(attacks_to_kill)
        return attacks_to_kill
    
    def models_to_kill(self, defender, is_ranged = False, s_ranged = 0.0, w_rnged = 0.0, ap_rng = 0.0):
        matk = self.mean_attacks_to_kill(defender, is_ranged, s_ranged, w_rnged, ap_rng)
        mtk = int(math.ceil(matk / self.a))
        print "Number of " + str(self.name) + " models requied to kill one " + str(defender.name) + " in one turn is " + str(mtk)
        return mtk

In [ ]:
wych_in_combat = Stats(name = "Wych", ws = 3, bs = 3, s = 3, t = 3, w = 1, a = 2, sv = 6)
wych_in_combat.set_inv(4)

necron_warrior = Stats(name = "Necron Warrior", ws = 3, bs = 3, s = 4, t = 4, w = 1, a = 1, sv = 4)

In [ ]:
wych_in_combat.models_to_kill(necron_warrior)